In [ ]:
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')


from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.options import Options
import pandas as pd
from google.colab import drive
from selenium.webdriver.support.ui import Select
import time
import requests
from bs4 import BeautifulSoup
import numpy as np

     |████████████████████████████████| 958 kB 5.1 MB/s 
     |████████████████████████████████| 138 kB 57.5 MB/s 
     |████████████████████████████████| 356 kB 49.8 MB/s 
     |████████████████████████████████| 55 kB 3.3 MB/s 
     |████████████████████████████████| 3.6 MB 45.2 MB/s 
     |████████████████████████████████| 54 kB 3.3 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.7 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-

/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.7) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [ ]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
browser = webdriver.Chrome(executable_path='chromedriver',options=chrome_options)
url = 'https://www.imdb.com/'
browser.get(url)
browser.find_element_by_xpath('(//div[@class="ipc-button__text" ])[2]').click()
time.sleep(2)
browser.find_element_by_link_text('Advanced Search').click()
browser.find_element_by_link_text('Advanced Title Search').click()
browser.find_element_by_id('title_type-1').click()
browser.find_element_by_id('title_type-2').click()
# released dates
released_date_min = browser.find_element_by_name('release_date-min')
released_date_min.click()
released_date_min.send_keys('1999')
released_date_max = browser.find_element_by_name('release_date-max')
released_date_max.click()
released_date_max.send_keys('2021')
# ratings
ratings_min = browser.find_element_by_name('user_rating-min')
dropdown = Select(ratings_min)
dropdown.select_by_visible_text('4.5')
ratings_max = browser.find_element_by_name('user_rating-max')
dropdown2 = Select(ratings_max)
dropdown2.select_by_visible_text('10')
# colour
browser.find_element_by_id('colors-1').click()
# language
lang = browser.find_element_by_name('languages')
dropdown3 = Select(lang)
dropdown3.select_by_visible_text('English')
# display
display = browser.find_element_by_id('search-count')
dropdown4 = Select(display)
dropdown4.select_by_index(2)
# submit
browser.find_element_by_xpath('(//button[@type="submit" ])[2]').click()
Movie_Title = []
Year = []
Ratings = []
Genre = []
Description = []
Director = []
Cast = []
l = 1
length = browser.find_element_by_xpath('//div[@class="desc"]/span').text.split('of')[1].replace('titles.','').replace(',','').strip()
no = round((int(length)/250))
while l <= (no-1):
    page = browser.current_url
    response = requests.get(page)
    soup = BeautifulSoup(response.content,'html.parser')
    list_items = soup.find_all('div',{'class':'lister-item'})
    for result in list_items:
        # movie title
        title_ = result.find('h3').find('a').text
        Movie_Title.append(title_)
        # year
        year_ = result.find('h3').find('span',{'class':'lister-item-year'}).text.replace('(','').replace(')','')
        Year.append(year_)
        # rating
        rating_ = result.find('div',{'class':'ratings-imdb-rating'}).text.strip()
        Ratings.append(rating_)
        # genre
        try:
            genre_ = result.find('span',{'class':'genre'}).text.strip()
        except AttributeError:
            genre_ = np.NAN
#         else:
#             genre_ = genre_
        Genre.append(genre_)
        # movie description
        description_ = result.find('div',{'class':'lister-item-content'}).find_next('p').find_next('p').text.strip()
        Description.append(description_)
        # director
        director_ = result.find('p').find_next('p').find_next('p').text.replace('\n','').split('|')[0].replace('Director:','')
        Director.append(director_)
        # cast
        try:
          cast_ = result.find('p').find_next('p').find_next('p').text.replace('\n','').split('|')[1].replace('Stars:','').strip()
        except IndexError:
          cast_ = np.NAN
        Cast.append(cast_)

    browser.find_element_by_class_name('next-page').click()
    l+=1
# converting to dataframe
movie_df = pd.Dataframe({'Movie_Title':Movie_Title,'Year':Year,'Ratings':Ratings,'Genre':Genre,'Description':Description,'Director':Director,'Cast':Cast})
drive.mount('/drive')
movie_df.to_csv('/drive/My Drive/Movies_list.csv')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  del sys.path[0]
/usr/local/lib/python3.7/dist-pack

NoSuchElementException: ignored

In [ ]:
len(Cast)

60996

In [ ]:
movie_df = pd.DataFrame({'Movie_Title':Movie_Title,'Year':Year,'Ratings':Ratings,'Genre':Genre,'Description':Description,'Director':Director,'Cast':Cast})
drive.mount('/drive')
movie_df.to_csv('/drive/My Drive/Movies_list.csv')

Mounted at /drive
